Entonces, ya viendo que funciona (al menos en este ejemplo tan sencillito), vale la pena ver que lo que está pasando dentro de la función symbolic_regression.

Vemos que recibimos un número grande de parámetros que explicaremos mientras se vayan usando para entender mejor.

Luego se incia el temporizador que será utilizado para comprobar cuánto tiempo tomó la ejecución de todo el algoritmo. Se inicializa la semilla que se utilizará en los "randoms" de la ejecución.

En features_names se almacenan los nombres de las variables que pueden aparecer en cada ecuación del sistema, o sea una una lista de listas en caso de que se pase en los parámetros, en caso de que este no se envíe pues se utilizan todos los parámetros en todas las ecuaciones. Esto es fundamentalmente para ser utilizado si conocemos información del sistema que se desea encontrar.

En operations se guardan las operaciones que pueden aparecer dentro de las ecuaciones del sistema.

A continuación se crea la población inicial del algoritmo genérico. Para esto se utiliza la función random_system que crea sistemas de ecuaciones aleatorios utilizando las variables en cada ecuación definidas anteriormente, junto con las operaciones, además se hace uso del parámetro MAX_DEPTH para controlar la profundidad que pueden tener estos árboles. La población inicial generada contiene POP_SIZE individuos.

Entonces inciamos las generaciones. Definida la máxima cantidad que pueden ocurrir de estas por el parámtro MAX_GENERATIONS.

Dentro de cada generación se toma una cantidad de individuos igual a MUTATION_SIZE para ser mutados, resaltar que no es que se muten estos encima del mismo sistema, si no que se crea otro con el resultado de la mutación. Estos individuos mutados son guardados en mutations_populations.

Lo que hizo esta función fue seleccionar aleatoriamente individuos de la población y aplicar la función mutate_system sobre ellos. Esta función selecciona un nodo aleatorio dentro del árbol representante del sistema y editar alguna de sus carácterísticas. Estas mutaciones pueden ir desde eliminar un subárbol a sencillamente cambiar la operación en un nodo.

Luego se realiza un proceso similar pero esta vez en vez de mutar a individuos, se cruzan entre ellos. Se cruzan una cantidad de 2 * XOVER_SIZE, para obtener un total de XOVER_SIZE individuos resultantes de cruzamientos. La función xover lo que hace es tomar dos sistemas de ecuaciones y devolver uno nuevo con características en común de ambos padres.

Entonces podemos generar una nueva población que contiene a la población incial, los individuos mutados y los individuos resultantes de los cruzamientos. Esta nueva población es guardada en total_population.

De estos sistemas no conocemos mucho, y necesitamos filtrarlos para quedarnos con una cantidad igual a la población inicial, además de ver que tan bueno fue el mejor de la población generada.

Para esto se recorren todos los individuos optimizando sus parámetros para que se ajusten al conjunto de datos. Esto se hace llamando a la función lineal_optimization_system que esta genera un sistema de ecuaciones lineales por cada una de las ecuaciones del sistema optimizando los parámetros y asignándolos en el sistema. Una vez los parámetros han sido ajustados se evalua la calidad del sistema con la función compute_fitness, que esta compara los resultados obtenidos en las evaluaciones del sistema y los resultados objetivos. Mientras este resultado sea lo más cercano a 0, quiere decir que la diferencia entre los resultados es cada menor, por lo tanto es mejor.

Si en algún momento el resultado obtenido es menor que un EPSILON definido en los parámetros se detiene la ejecución. Si esto no ocurre pues pasan a la siguiente generación los individuos que mejor puntuación obtuvieron, una cantidad de estos igual a POP_SIZE.

Antes de retornar finalmente el sistema obtenido, se redondean los parámetros y se filtran aquellos que son 0 para "embellecer" el sistema



